In [ ]:
# Install python-telegram-bot
%pip install python-telegram-bot --upgrade

In [ ]:
# Clone the master branch of the Tensorflow Models repository
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Import the lastest version of Tensorflow
import tensorflow as tf

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Mount the google drive if you are working with cloud drive. Comment out this section if you working with local file system
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Test the installation
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging
from datetime import datetime
import numpy as np
from six import BytesIO
from collections import defaultdict
from io import StringIO
from PIL import Image
import time
# Import utilites
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils

# Type your Telegram bot token
bot = telegram.Bot(token='xx')

updater = Updater(token='xx', use_context=True)

dispatcher = updater.dispatcher

# Log the activity of this bot by .log file. Type your desired name for the .log file.
logging.basicConfig(filename='/content/drive/MyDrive/xx/yy.log',format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

# Patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Load the inference graph
masking_model = tf.saved_model.load('/content/drive/MyDrive/xx/saved_model')

# Load the labelmap
PATH_TO_LABELS = '/content/drive/MyDrive/xx.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Type your desired resolution for resizer 
width = 1440
height = 1920
dim = (width, height)

def start(update, context):
    # Type your Welcome message here
    context.bot.send_message(chat_id=update.effective_chat.id, text="Send me an image like this and the bot will count the needles for you!\nPlease send the pictures in original files for higher image quality."
                            )
    # Place your example detection here
    context.bot.send_photo(chat_id=update.effective_chat.id, photo=open('/content/drive/MyDrive/xx.jpg', 'rb'))

def detection_image(update, context):
    photos = update.message.photo
    documents = update.message.document
    # Timestamp in filename
    now = datetime.now()
    timestamp_date = now.strftime("%Y%m%d")
    timestamp_time = now.strftime("%H%M%S")
    if photos:
        user = update.message.from_user
        update.message.reply_text("I have received your image. Now detecting needles..\nIt may take 10 seconds to process.. ")
        logger.info(f"Photo received from {user.first_name} {user.last_name}")
        start_time = time.time()
        # Download the 1280x960 photo and save the image in your desired cloud drive directory
        photo_id = photos[-1].file_id
        context.bot.get_file(photo_id).download(f"/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}.jpg")
        image_path = f'/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}.jpg' 
    elif documents.mime_type == 'image/heic' or 'image/jpeg' or 'image/png':
        user = update.message.from_user
        update.message.reply_text("I have received your image. Now detecting needles..\nIt may take 10 seconds to process.. ")
        logger.info(f"H-res Photo received from {user.first_name} {user.last_name}")
        start_time = time.time()
        # Download the uncompressed photo and save the image in your desired cloud drive directory
        context.bot.get_file(documents.file_id).download(f"/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}.jpg")
        hres = Image.open(f"/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}.jpg")
        hres_resized = hres.resize (dim)
        hres_resized.save(f"/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}_resized.jpg")
        image_path = f'/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}_resized.jpg'

    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = masking_model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop("num_detections"))

    # detections = {key: value[0, :num_detections].numpy()
    #             for key, value in detections.items()}

    import itertools
    detections = dict(itertools.islice(detections.items(), num_detections))

    detections["num_detections"] = num_detections

    image_np_with_detections = image_np.copy()

    # Handle models with masks:
    if "detection_masks" in detections:
      # Reframe the the bbox mask to the image size.
      detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
          detections["detection_masks"][0], detections["detection_boxes"][0],
           image_np.shape[0], image_np.shape[1])      
      detection_masks_reframed = tf.cast(detection_masks_reframed > 0.1,
                                   tf.uint8)
      detections["detection_masks_reframed"] = detection_masks_reframed.numpy()

    boxes = np.asarray(detections["detection_boxes"][0])
    classes = np.asarray(detections["detection_classes"][0]).astype(np.int64)
    scores = np.asarray(detections["detection_scores"][0])
    #mask = np.asarray(detections["detection_masks_reframed"])

    # Count the total number of detections made.
    final_score = np.asarray(detections["detection_scores"][0])
    count = 0
    for i in range(100):
      if scores is None or final_score[i] > 0.9:
        count = count + 1

    # Visualizing the results
    vis_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        boxes,
        classes,
        scores,
        category_index,
        #instance_masks=mask,
        use_normalized_coordinates=True,
        line_thickness=1,
        max_boxes_to_draw=30,
        min_score_thresh=0.9,
        )
    update.message.reply_text(f"Needle count:{count}")
    # Save the detection result in your desired directory
    fimg = Image.fromarray(image_np_with_detections)
    fimg.save(f'/content/drive/MyDrive/zz/{count}-{timestamp_date}-{timestamp_time}.jpg')
    update.message.reply_photo(open(f'/content/drive/MyDrive/zz/{count}-{timestamp_date}-{timestamp_time}.jpg','rb'))
    end_time = time.time()
    elapsed_time = end_time - start_time
    logger.info(f"Photo sent to {user.first_name} {user.last_name}, elapsed time:{elapsed_time}")
    # Rename images for easy reference
    OLD_DOWNLOAD = f'/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}.jpg'
    NEW_DOWNLOAD = f'/content/drive/MyDrive/yy/{timestamp_date}-{timestamp_time}_{user.first_name}_{user.last_name}.jpg'
    OLD_PROCESS = f'/content/drive/MyDrive/zz/{count}-{timestamp_date}-{timestamp_time}.jpg'
    NEW_PROCESS = f'/content/drive/MyDrive/zz/{count}-{timestamp_date}-{timestamp_time}_{user.first_name}_{user.last_name}.jpg'
    os.renames (OLD_DOWNLOAD,NEW_DOWNLOAD)
    os.renames (OLD_PROCESS,NEW_PROCESS)
        
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)
dispatcher.add_handler(MessageHandler(Filters.photo | Filters.document, detection_image))

updater.start_polling()

        

In [ ]:
# Stop the telegram bot
updater.stop()